<div style="border-style: solid;border-width: 1px;">
<div style="height: 200px; 
            width: 750px; 
            color: #ffffff;
            margin: auto;
	        background-color: #ff6600; 
	        background-image: url(http://icarus.lngs.infn.it/img/n3.jpg); 
            background-repeat: no-repeat;
            background-position: center;">
</div>

<div style="text-align:center">
    <h1><font color="blue"><font size="6">Analysis of Noise Filtering Techniques in ICARUS</font></font></h1><br>
    <font color="gray"><font size="3">A jupyter notebook aimed at studying Noise Filtering with the ICARUS LAr TPC</font></font><br>
</div>
</div>
<br>
<h3>What this notebook intends to do:</h3>
<ul>
    <li>Operates on art root format data files that contain raw::RawDigit objects</li>
        <ul>
            <li>The RawDigits contain waveforms representing the TPC readout </li>
        </ul>
    <li>Once the RawDigits have been recovered the idea is to analyze the raw data</li>
        <ul>
            <li>Understand impacts of signal degradation on coherent noise removal</li>
            <li>Look at various algorithms for reducing white noise while maintaining signal</li>
        </ul>
</ul>

<h3>What is needed to run this notebook:</h3>
<ul>
    <li><p>Necessary python libraries/includes</p>
        <ul>
            <li> matplotlib   - the basic plotting package though note that here we mostly use plot-ly</li>
            <li> numpy        - python array handling and loads of useful functions</li>
            <li> scipy        - where we get the analytic functions we will make heavy use of </li>
            <li> ipywidgets   - useful for making interactive output</li>
            <li> uproot       - see below</li>
            <li> plotly       - the plot-ly graphing package which allows interactive plots</li>
        </ul></li>
    <li>uproot<br>
        In this notebook we use "uproot" to handle the root input. For more information on uproot follow this <a href="https://uproot.readthedocs.io/en/latest/">link</a></li>
</ul>



<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Start Up Block</font></font></h2><br>
</div>

<ul>
    <li>Import of standard libraries used below</li>
    <li>Set the system path to include looking in the package of noise handling functions/objects</li>
    <li>We define two variables which define the location of the input file</li>
        <ul>
            <li> PATHNAME       - Fully qualified path to the folder containing the input file</li>
            <li> RECOFILENAME   - the fully qualified file name for the input data file</li>
        </ul>
    <li>With these definitions we can open and read the input data file</li>
</ul>



In [ ]:
# Probably most people will simply want "matplotlib inline" but on a mac it is useful to do the other stuff
%matplotlib inline
##matplotlib qt
##matplotlib notebook
##config InlineBackend.figure_format ='retina'
#%pylab

# This allows reading of root files without having to actually use root. Nice!
import uproot

# Widgets are always good for you
#from ipywidgets import widgets,Layout

###############################################################################
# TODO:
# Set the path to our library of functions/objects/etc
#import sys

#sigProcPath = "/home/usher/LArTPC/icarus-sigproc-tools"
#sys.path.insert(0,sigProcPath)
###############################################################################
# TODO:
# Set the path and name of the data file to be read
PATHNAME       = "/home/usher/LArTPC/ICARUS/workarea/"
RECOFILENAME   = PATHNAME + "Supplemental-prod_muon_0-2GeV_isotropic_icarus_20191113T152944_gen_20191113T154101-G4_20191129T185220-DetSimTest_20191201T171339-McRecoGauss.root"


<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Open and Read the Input File</font></font></h2><br>
</div>

<ul>
    <li>We define three variables which should be common to all files
        <ul>
            <li> RECOFOLDERNAME - the name of the folder for the particular data set we want to analyze</li>
            <li> DAQNAME.       - currently unused</li>
        </ul></li>
    <li>With these definitions we can open and read the input data file</li>
    <li>Contents of the input file are read into a numpy array</li>
</ul>



In [ ]:
# Grab the pandas dataframe from the input file for the tree we want to look at
print("Opening file: ",RECOFILENAME)
data_file = uproot.open(RECOFILENAME)

print(data_file.keys(),"\n------\n")

HISTOGRAMNAME = "gaushitTPC2"

print("Opening the folder containing the hists folder information: ",HISTOGRAMNAME)
hist_folder = data_file[HISTOGRAMNAME]

print(hist_folder.keys(),"\n------\n")

EVENTFOLDER = "Event0000"

print("Opening the folder containing the hists folder information: ",EVENTFOLDER)
event_folder = hist_folder[EVENTFOLDER]

print(event_folder.keys(),"\n------\n")

PLANEFOLDER = "c1T0P0"

print("Opening the folder containing the plane hists folder information: ",PLANEFOLDER)
plane_folder = event_folder[PLANEFOLDER]

print(plane_folder.keys(),"\n------\n")

WIREFOLDER = "Wire_00525"

print("Opening the folder containing the wire folder information: ",WIREFOLDER)
wire_folder = plane_folder[WIREFOLDER]

print(wire_folder.keys(),"\n------\n")

#WAVEFORM = "HWfm_001_roiStart-00798"
#
#print("Opening the folder containing the waveform hist information: ",WAVEFORM)
#waveform = wire_folder[WAVEFORM]
#
#print(type(waveform))
#print(waveform,"\n------\n")
#print(waveform.edges)
#print(waveform.values)


In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as subplots
from plotly.colors import DEFAULT_PLOTLY_COLORS

# These will be useful
waveformX   = np.arange(0,4096)

yValueLabel = ['Waveform','Derivative','Erosion','Dilation','Candidate','Maximum','SSS']

# Define a function which will produce a single waveform for each of the snippets in a given wire's folder
def plotWaveforms(wire_folder):
    yValueDict  = {'HWfm' : np.zeros(4096),
                   'HDer' : np.zeros(4096),
                   'HEro' : np.zeros(4096),
                   'HDil' : np.zeros(4096),
                   'HCan' : np.zeros(4096),
                   'HMax' : np.zeros(4096),
                   'HSSS' : np.zeros(4096)
                  }
    
    # First loop is to place the individual snippets into a full waveform
    for keyBytes in wire_folder.keys():
        key        = str(keyBytes)
        trimmedKey = key[key.index('H'):key.index(';')]
        dictKey    = trimmedKey[0:trimmedKey.index('_')]
        startTick  = int(key[key.index('-')+1:key.index(';')])
        adcValues  = wire_folder[trimmedKey].values
        yValueDict[dictKey][startTick:startTick+len(adcValues)] = adcValues
    
    colorIdx = 0
    plotList = []
    
    # Now go through and produce the scatter plots
    for key in yValueDict:
        plotList.append(
            go.Scatter(
                visible=False,
                line=dict(color=DEFAULT_PLOTLY_COLORS[colorIdx], width=1),
                name=yValueLabel[colorIdx],
                x=waveformX,
                y=yValueDict[key]))
        colorIdx += 1
        
    return plotList
    
# Define a function to produce scatter plots for all of the wires in a given plane
def plotWires(plane_folder):
    labelList = []
    plotList  = []
    for keyBytes in plane_folder.keys():
        key        = str(keyBytes)
        trimmedKey = key[key.index('W'):key.index(";")]
        labelList.append(trimmedKey[trimmedKey.index("_")+1:].lstrip('0'))
        plotList += plotWaveforms(plane_folder[trimmedKey])
        
    return labelList,plotList

selectedPlane = 0

# Loop over the planes in the event folder
# This will be all channels in a single TPC for a given event
for keyBytes in event_folder.keys():
    key        = str(keyBytes)
    trimmedKey = key[key.index("c"):key.index(";")]
    planeNum   = int(trimmedKey[-1])
    
    if planeNum != selectedPlane:
        continue;

    # Get the plot information
    labelList,plotList = plotWires(event_folder[trimmedKey])
    nWires             = len(labelList)
    
    waveformFig = go.Figure(data=plotList)
    
    # Make first trace visible
    nHistogramsPerWire = len(plotList)//nWires
    
    for idx in range(nHistogramsPerWire):
        waveformFig.data[idx].visible = True
    
    # Create and add slider
    steps = []
    for i in range(nWires):
        step = dict(
            method="restyle",
            args=["visible", [False] * len(plotList)],
            label=labelList[i]
        )
        step["args"][1][i*nHistogramsPerWire:(i+1)*nHistogramsPerWire] = [True] * nHistogramsPerWire  # Toggle i'th histograms to "visible"
        steps.append(step)
    
    slider = [dict(
        active=0,
        currentvalue={"prefix": "Wire: "},
        #xanchor="left",
        #yanchor="auto",
        #y=sliderY[planeNum],
        #transition={"duration",200},
        pad={"t": 20},
        steps=steps
    )]

    waveformFig.update_layout(
        sliders=slider,
        #height=400,
        #width=900,
        title_text="Deconvolved Waveforms Plane "+str(planeNum),
        title_font_size=25
    )
    
    waveformFig.show()


In [ ]:
import sys

sigProcPath = "/home/usher/LArTPC/jupyter/icarus-sigproc-tools"
sys.path.insert(0,sigProcPath)

from sigproc_tools.sigproc_objects.filter import FilterInduction0

numTimeBins  = 4096
samplingRate = 0.4
#
## local constants
#samplingRate = 0.4                                 # 0.4 us per tick
#maxFrequency = 1000. / (2.*samplingRate)           # max sampling frequency in MHz
#frequencyRes = maxFrequency / float(numTimeBins/2) # frequency resolution in MHz
#
#frequencyBins = np.arange(0.,maxFrequency,frequencyRes)
#
#print(frequencyBins)
#
#params1 = (0.,4.0)
#params2 = (0.,100.)
#
#gauss1 = np.exp(-(frequencyBins-params1[0])/(2.*params1[1]*params1[1]))
#gauss2 = np.ones(len(frequencyBins)) - np.exp(-(frequencyBins-params2[0])/(2.*params2[1]*params2[1]))
#
#print(gauss1)
#print(gauss2)
#
#filter = gauss1 * gauss2
#
#print(filter)
#
#largestElem = np.amax(filter)
#
#print(largestElem)
#
#filter /= largestElem

filterInduction0 = FilterInduction0(numTimeBins,samplingRate)

gaussPlot = go.Figure()

gaussPlot.add_trace(
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[0], width=1),
        name="gauss plot",
        x=filterInduction0.frequencyBins,
        y=filterInduction0.filter))

gaussPlot.show()